In [63]:
class Tree:

    def __init__(self, name='root', classifier = None):
        self.children = []
        self.name = name
        self.classifier = classifier
        self.data = {}

    def set_data(self, data):
        self.data = data

    def set_name(self, name):
        self.name = name

    def set_classifier(self, classifier):
        self.classifier = classifier

    def __repr__(self):
        return str(self.name)

    def add_child(self, tree):
        self.children.append(tree)

    def remove_child(self, child):
        self.children.remove(child)

    def clear_children(self, tree):
        self.children = []

    def isLeaf(self):
        return self.children.count(None) == len(self.children)

    def show(self, ind = 0):
        indent = ''
        for i in range(ind):
            indent = indent + ' | '

        if self.isLeaf():
            print(indent, self.name, '-->', self.classifier, self.data)
        else:
            print(indent, self.name, 'is a parent with children:',self.children , self.data)

            for child in self.children:
                child.show(ind + 1)

____


In [64]:
import numpy as np
from sklearn.model_selection import train_test_split
from Tree import Tree
import operator

def learn(X, y, impurity_measure = 'entropy', pruning = False):
    if len(y) == 0:
        return 0

    (X,y) = update_data(X, y)
    print('Impurity_measure:', impurity_measure)

    #X = X_train, y = y_train
    #if pruning:
    #    tree = makeTree(X_train, y_train, impurity_measure)

    return makeTree(X, y, impurity_measure)

In [65]:
def update_data(X, y):
    newX = []
    newy = []
    for row in range(len(y)):
        if '?' not in X[row]:
            newX.append(X[row])
            newy.append(y[row])
    return (newX, newy)

In [66]:
def predict(x, tree):
    if tree.isLeaf():
        return tree.classifier
    else:
        for child in tree.children:
            child.name = child.name.strip()
            if child.name in x:
                list(x).remove(child.name)
                return predict(x, child)

In [67]:
def makeTree(X, y, impurity_measure):
    if is_pure(y):
        return Tree(classifier = y[0])

    elif len(np.transpose(X)) == 0: # no features left
        mcl = most_common_label(y)
        return Tree(classifier = mcl)

    else:
        tree = Tree()
        index = calculateInformationGain(X, y, impurity_measure)

        for attribute_value, [splitted_X, splitted_y] in split(X, y, index).items():
            child = makeTree(splitted_X, splitted_y, impurity_measure)
            child.set_name(attribute_value)
            child.set_data(countLetters(splitted_y))
            tree.add_child(child)

    return tree

This function is used to split after finding the best index in data X. 
    X:
        Data X (may also be a new X, after getting splitted) 
    
    y:
        Data y (may be new y, after getting splitted)

    index:
        Index after finding the best feature (index of a column in X) 

    return dict:
        Returns a dictionary with attribute and a corresponding classifier. 

In [3]:
def split(X, y, index):
    dict = {}
    for i in range(len(y)):
        if X[i][index] in dict:
            dict[X[i][index]][0].append(X[i][:index] + X[i][index+1:])
            dict[X[i][index]][1].append(y[i])
        else:
            dict[X[i][index]] = [[X[i][:index] + X[i][index+1:]], [y[i]]]
    return dict

Function is_pure() checks if a dataset only contains the value
    y: 
        A dataset 
    
    return len(set(y)) == 1:
        Returns either True or False depending on the data

Function most_common_label() see which attribute is the most common in the dataset
    y: 
        A dataset

    return sortedClassifier[0][0]:
        Returns the most common label (attribute) in the dataset

In [4]:
def is_pure(y):
    return len(set(y)) == 1

def most_common_label(y):
    dict = {}
    for classifier in y:
        if classifier not in dict.keys():
            dict[classifier] = 0
        dict[classifier] += 1
    sortedClassifier = sorted(dict.items(), key = operator.itemgetter(1), reverse=False)
    return sortedClassifier[0][0]

In [5]:
def calculateInformationGain(X, y, impurity_measure):
    ig_list = []
    #A dictionary with function mapped to the keys
    impurity_func = {'entropy': calc_entropy, 'gini': calc_gini}
    #measure is a function that matches the impurity_measure 
    measure = impurity_func.get(impurity_measure)

    for row in np.transpose(X):
        #Put the probabilities of the values in a list 
        probabilities = [counter/len(y) for counter in countLetters(y).values()]
        
        #Here we calculate the probability 
        information_measure = measure(probabilities)
        ig = information_measure
        for attribute_value, occurrence_dict in zip_xy_class(row, y).items():
            s = sum(occurrence_dict.values())
            X_probabilities = [counter/s for counter in occurrence_dict.values()]
            attribute_measure = measure(X_probabilities)
            weight = s/len(y)

            gain -= weight * attribute_measure

        ig_list.append(gain)

    index = np.argmax(ig_list)
    return index

In [6]:
def calc_entropy(listprob):
    entropy = 0
    for prob in listprob:
        if prob != 0:
            entropy += -prob * np.log2(prob)
    return entropy

def calc_gini(listprob):
    gini = 0
    for prob in listprob:
        if prob != 0:
            gini += prob * (1 - prob)
    return gini

In [7]:
def countLetters(array):
    dict = {}
    for i in array:
        if i in dict:
            dict[i] += 1
        else:
            dict[i] = 1
    return dict

In [8]:
def zip_xy_class(X, y):
    dict = {}
    for attribute, classifier in list(zip(X, y)):
        if attribute in dict:
            if classifier in dict[attribute]:
                dict[attribute][classifier] += 1
            else:
                dict[attribute][classifier] = 1
        else:
            dict[attribute] = {classifier : 1 }
    return dict

 
 
 
 
 ____


In [1]:
import csv
import numpy as np
from ID3 import learn, makeTree, predict
from Tree import Tree

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

X = []
y = []

with open('agaricus-lepiota.data') as csv_file:
        reader = csv.reader(open("agaricus-lepiota.data", "rb"), delimiter=",")
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            y.append(row[0])
            X.append(row[1:])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)


#For my ID3 implementation:
myTree = learn(X_train, y_train, 'entropy', True)
myTree.show()

dict = {}
for row in range(len(X_test)):
    pred = predict(X[row], myTree)
    result = pred == y_test[row]
    if result not in dict:
        dict[result] = 1

    if (result):
        dict[result] += 1
    else:
        dict[result] += 1
print(dict)

#print(prune(X, y, tree))

#5. Implementation comparision
X_T = []
le = LabelEncoder()
for i in range(len(np.transpose(X_train))):
    X_T.append(le.fit_transform(np.transpose(X_train)[i]))
X_train = np.transpose(X_T)

X_T = []
for i in range(len(np.transpose(X_train))):
    X_T.append(le.fit_transform(np.transpose(X_test)[i]))
X_test = np.transpose(X_T)

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)


dtc = tree.DecisionTreeClassifier(criterion = 'entropy')
dtc.fit(X_train, y_train)
dtc_predict = dtc.predict(X_test)

dict = {}
for i in dtc_predict:
    result = dtc_predict[i] == y_test[i]
    if result not in dict:
        dict[result] = 1

    if (result):
        dict[result] += 1
    else:
        dict[result] += 1

print(dict)






Impurity_measure: entropy
 branch is a parent with children: [n, c, f, l, a, p, m] {}
 |  n is a parent with children: [k, n, w, r] {'e': 1825, 'p': 59}
 |  |  k --> e {'e': 865}
 |  |  n --> e {'e': 896}
 |  |  w is a parent with children: [p, y, n, c, g, w] {'e': 64, 'p': 12}
 |  |  |  p --> e {'e': 4}
 |  |  |  y --> p {'p': 7}
 |  |  |  n --> e {'e': 31}
 |  |  |  c --> e {'e': 24}
 |  |  |  g --> e {'e': 5}
 |  |  |  w --> p {'p': 5}
 |  |  r --> p {'p': 47}
 |  c --> p {'p': 128}
 |  f --> p {'p': 1053}
 |  l --> e {'e': 253}
 |  a --> e {'e': 264}
 |  p --> p {'p': 162}
 |  m --> p {'p': 26}
{False: 1740, True: 943}
{True: 2682}


1. Implement the ID3 algorithm from scratch
The implementation is above

2. Gini Index
The learn() has entropy as a default impurity measure, but can be switched to gini with giving 'gini' as parameter

3. Pruning
Did not implement the pruning because of time remaining. 

4. Classify edible and poisonous mushrooms
Entropy and gini gives the same result of splitting even with random state. 

5. Implementation comparison:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

Splitted the data into training data and test data with test_size 0.33

For my implementation, my prediction got {False: 1740, True: 943} while the sklearn got {True: 2682}. 

My predict() is not completely correct, because I had problems with removing an attribute from the data I want to predict. Did not find another solution for this because of the time remaining. 
